In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dataset=pd.read_csv("AirPassengers.csv")

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
#convert string format into datetime format
dataset['Month']=pd.to_datetime(dataset['Month'],infer_datetime_format=True)

In [ ]:
indexed_data=dataset.set_index(['Month'])
indexed_data.head()

In [ ]:
 dataset['Month'].dtype

In [ ]:
plt.xlabel('Date')
plt.ylabel("NO.of passengers")
plt.plot(indexed_data)

In [ ]:
#DEtermine rolling statistics
rolmean=indexed_data.rolling(window=12).mean()
rolstd=indexed_data.rolling(window=12).std()

In [ ]:
print(rolmean,rolstd)

In [ ]:
rolmean.head(12)

In [ ]:
original=plt.plot(indexed_data,color='blue',label='original')
mean=plt.plot(rolmean,color='red',label='rolmean')
std=plt.plot(rolstd,color='black',label='rolstd')
plt.legend(loc='best')
plt.title('rolling mean and std')
plt.show(block=False)

In [ ]:
#Dicky_Fuller test
from statsmodels.tsa.stattools import adfuller
print('results of dickey test:')
dftest=adfuller(indexed_data["#Passengers"],autolag='AIC')
dfoutput=pd.Series(dftest[0:4],index=['Test statistic','P value','lag values','num of obs'])
for key,value in dftest[4].items():
    dfoutput['Critical value%key']=value
print(dfoutput)

In [ ]:
indexeddata_log=np.log(indexed_data)
plt.plot(indexeddata_log)

In [ ]:
#series stationaey
mvgAvg=indexeddata_log.rolling(window=12).mean()
mvgStd=indexeddata_log.rolling(window=12).std()
plt.plot(indexeddata_log)
plt.plot(mvgAvg,color='gray')
#dataset-mvgavg=indexeddata_log-mvgavg
#dataset.head()

In [ ]:
dataset_mvgAvg = indexeddata_log - mvgAvg
dataset_mvgAvg.head(12)


In [ ]:
#remove nullvalues
dataset_mvgAvg.dropna(inplace=True)
dataset_mvgAvg.head(10)

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    mvgAvg=timeseries.rolling(window=12).mean()
    mvgStd=timeseries.rolling(window=12).std()
    original=plt.plot(timeseries,color='blue',label='original')
    mean=plt.plot(mvgAvg,color='red',label='rolmean')
    std=plt.plot(mvgStd,color='black',label='rolstd')
    plt.legend(loc='best')
    plt.title('rolling mean and std')
    plt.show(block=False)      
    from statsmodels.tsa.stattools import adfuller
    print('results of dickey test:')
    dftest=adfuller(timeseries["#Passengers"],autolag='AIC')
    dfoutput=pd.Series(dftest[0:4],index=['Test statistic','P value','lag values','num of obs'])
    for key,value in dftest[4].items():
        dfoutput['Critical value%key']=value
    print(dfoutput)

In [ ]:
test_stationarity(dataset_mvgAvg)

In [ ]:
#exponential 
expdecaywtavg=indexeddata_log.ewm(halflife=12,min_periods=0,adjust=True).mean()
plt.plot(indexeddata_log)
plt.plot(expdecaywtavg)
dataexpdecaywtavg=indexed_data-expdecaywtavg
test_stationarity(dataexpdecaywtavg)

In [ ]:
datalogshift=indexeddata_log-indexeddata_log.shift()
plt.plot(datalogshift)
datalogshift.dropna(inplace=True)
test_stationarity(datalogshift)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomp=seasonal_decompose(indexeddata_log)

In [ ]:
trend=decomp.trend
sea=decomp.seasonal
resid=decomp.residplt.subplot(411)
plt.plot(indexeddata_log,label='original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend,label='trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(sea,label='seasonal')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(resid,label='resid')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
decomplog=resid
decomplog.dropna(inplace=True)
test_stationarity(decomplog)

In [ ]:
#ACF AND PACF
from statsmodels.tsa.stattools import acf,pacf
log_acf=acf(datalogshift,nlags=20)
log_pacf=pacf(datalogshift,nlags=20,method='ols')


In [ ]:
#plot acf
plt.subplot(121)
plt.plot(log_acf)
plt.axhline(y=0,linestyle="--",color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datalogshift)))
plt.axhline(y=0,linestyle="--",color='gray')
plt.axhline(y=1.96/np.sqrt(len(datalogshift)))
plt.title('AUTOCORRELATIONPLOT')
plt.subplot(122)
plt.plot(log_acf)
plt.axhline(y=0,linestyle="--",color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datalogshift)))
plt.axhline(y=0,linestyle="--",color='gray')
plt.axhline(y=1.96/np.sqrt(len(datalogshift)))
plt.title('AUTOCORRELATIONPLOT')
plt.tight_layout()



In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model=ARIMA(indexeddata_log,order=(2,1,2))
results_AR=model.fit(disp=-1)
plt.plot(datalogshift)
plt.plot(results_AR.fittedvalues,color='red')
plt.title('RSS: %.4f'%sum((results_AR.fittedvalues-datalogshift['#Passengers'])**2))
print('results')